In [209]:
# install.packages("xts")

In [210]:
# libraries
library(readxl)
options(warn = 0)
library(dplyr)
library(ggplot2)
library(caTools)
library(caret)
library(GGally)
library(janitor)
library(corrplot)
library(tidyverse)
library(hrbrthemes)
library(viridis)
library(neuralnet)
library(xts)

In [211]:
df <- read_excel("data/UOW_load.xlsx")
head(df)

Dates,09:00,10:00,11:00
<dttm>,<dbl>,<dbl>,<dbl>
2018-01-01,89.4,90.6,88.6
2018-01-02,108.2,104.6,106.0
2018-01-03,110.0,111.6,114.8
2018-01-04,106.4,104.4,109.0
2018-01-05,97.8,100.4,102.4
2018-01-06,87.0,90.8,87.2


In [212]:
df$Dates<-as.Date(df$Dates)

In [213]:
names(df)[2] <- paste("ninth_hour")
names(df)[3] <- paste("tenth_hour")
names(df)[4] <- paste("eleventh_hour")
head(df)

Dates,ninth_hour,tenth_hour,eleventh_hour
<date>,<dbl>,<dbl>,<dbl>
2018-01-01,89.4,90.6,88.6
2018-01-02,108.2,104.6,106.0
2018-01-03,110.0,111.6,114.8
2018-01-04,106.4,104.4,109.0
2018-01-05,97.8,100.4,102.4
2018-01-06,87.0,90.8,87.2


In [214]:
summary(df[2:4])

   ninth_hour       tenth_hour     eleventh_hour   
 Min.   : 50.40   Min.   : 49.20   Min.   : 48.20  
 1st Qu.: 81.55   1st Qu.: 83.00   1st Qu.: 84.75  
 Median : 97.50   Median :102.80   Median :105.40  
 Mean   : 95.35   Mean   : 99.63   Mean   :102.83  
 3rd Qu.:110.60   3rd Qu.:116.05   3rd Qu.:120.90  
 Max.   :141.20   Max.   :148.80   Max.   :156.40  

In [215]:
# insert all the inputs to the one data frame
df_full = df %>%
 mutate(
     previous_one_day_ninth_hour = lag(df$ninth_hour,1),
    previous_one_day_tenth_hour = lag(df$tenth_hour,1),
    previous_one_day_eleventh_hour = lag(df$eleventh_hour,1),
    seven_day_rolling = rollmean(eleventh_hour,7, fill = NA)) %>%
#Drop null coulumns
drop_na()


In [216]:
head(df_full)

Dates,ninth_hour,tenth_hour,eleventh_hour,previous_one_day_ninth_hour,previous_one_day_tenth_hour,previous_one_day_eleventh_hour,seven_day_rolling
<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2018-01-04,106.4,104.4,109.0,110.0,111.6,114.8,96.51429
2018-01-05,97.8,100.4,102.4,106.4,104.4,109.0,100.45714
2018-01-06,87.0,90.8,87.2,97.8,100.4,102.4,101.88571
2018-01-07,68.8,67.0,67.6,87.0,90.8,87.2,101.80000
2018-01-08,110.2,113.0,116.2,68.8,67.0,67.6,102.42857
2018-01-09,111.2,119.2,116.0,110.2,113.0,116.2,105.00000


In [217]:
dim(df_full)

[1] 494   8

In [218]:
z_score = function(x) {
  return((x - mean(x)) / sd(x))
}

x_value = function(z){
    return((z * sd(z)) + mean(z))
}

In [219]:
dfNorm <- as.data.frame(lapply(df_full[2:8], z_score))
dfNorm

ninth_hour,tenth_hour,eleventh_hour,previous_one_day_ninth_hour,previous_one_day_tenth_hour,previous_one_day_eleventh_hour,seven_day_rolling
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.5266947,0.20580282,0.24127518,0.6945286,0.51527414,0.46659127,-0.49882285
0.1185207,0.03213252,-0.01875840,0.5231038,0.20144026,0.23721832,-0.19247025
-0.3940699,-0.38467621,-0.61762362,0.1135891,0.02708811,-0.02379228,-0.08147293
-1.2578799,-1.41801451,-1.38984455,-0.4006851,-0.39135705,-0.62490760,-0.08813277
0.7070507,0.57919398,0.52494817,-1.2673325,-1.42875235,-1.40002999,-0.03929395
0.7545128,0.84838294,0.51706837,0.7040522,0.57629739,0.52195715,0.16050123
0.6500961,0.64866210,0.44615012,0.7516702,0.84654323,0.51404774,0.13386187
0.2134449,0.40552367,0.41463089,0.6469106,0.64603825,0.44286303,0.12498208
0.7450203,0.65734561,0.69042408,0.2088251,0.40194524,0.41122539,0.18048074


In [220]:
dfNorm <- as.data.frame(lapply(dfNorm[1:7],x_value))
dfNorm

ninth_hour,tenth_hour,eleventh_hour,previous_one_day_ninth_hour,previous_one_day_tenth_hour,previous_one_day_eleventh_hour,seven_day_rolling
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.5266947,0.20580282,0.24127518,0.6945286,0.51527414,0.46659127,-0.49882285
0.1185207,0.03213252,-0.01875840,0.5231038,0.20144026,0.23721832,-0.19247025
-0.3940699,-0.38467621,-0.61762362,0.1135891,0.02708811,-0.02379228,-0.08147293
-1.2578799,-1.41801451,-1.38984455,-0.4006851,-0.39135705,-0.62490760,-0.08813277
0.7070507,0.57919398,0.52494817,-1.2673325,-1.42875235,-1.40002999,-0.03929395
0.7545128,0.84838294,0.51706837,0.7040522,0.57629739,0.52195715,0.16050123
0.6500961,0.64866210,0.44615012,0.7516702,0.84654323,0.51404774,0.13386187
0.2134449,0.40552367,0.41463089,0.6469106,0.64603825,0.44286303,0.12498208
0.7450203,0.65734561,0.69042408,0.2088251,0.40194524,0.41122539,0.18048074


In [221]:
dim(df_full)

[1] 494   8

In [222]:
head(df_full)

Dates,ninth_hour,tenth_hour,eleventh_hour,previous_one_day_ninth_hour,previous_one_day_tenth_hour,previous_one_day_eleventh_hour,seven_day_rolling
<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2018-01-04,106.4,104.4,109.0,110.0,111.6,114.8,96.51429
2018-01-05,97.8,100.4,102.4,106.4,104.4,109.0,100.45714
2018-01-06,87.0,90.8,87.2,97.8,100.4,102.4,101.88571
2018-01-07,68.8,67.0,67.6,87.0,90.8,87.2,101.80000
2018-01-08,110.2,113.0,116.2,68.8,67.0,67.6,102.42857
2018-01-09,111.2,119.2,116.0,110.2,113.0,116.2,105.00000


In [223]:
df_train=df_full[1:400,]
df_test  =df_full[401:nrow(df_full),]

In [224]:
# fit neural network
nn=neuralnet(eleventh_hour~ninth_hour+tenth_hour,data=df_train[,c(1,2,3)], 
            hidden=10,
            linear.output = TRUE)

ERROR: Error in eval(predvars, data, env): object 'eleventh_hour' not found


In [ ]:
plot(nn)

In [ ]:
summary(nn)

                    Length Class      Mode    
call                  5    -none-     call    
response            400    -none-     numeric 
covariate           800    -none-     numeric 
model.list            2    -none-     list    
err.fct               1    -none-     function
act.fct               1    -none-     function
linear.output         1    -none-     logical 
data                  3    data.frame list    
exclude               0    -none-     NULL    
net.result            1    -none-     list    
weights               1    -none-     list    
generalized.weights   1    -none-     list    
startweights          1    -none-     list    
result.matrix        44    -none-     numeric 

In [ ]:
## Prediction using neural network
Predict=compute(nn,df_test[,c(1,2)])
# Predict$net.result

In [ ]:
str(Predict)

List of 2
 $ neurons   :List of 2
  ..$ : num [1:94, 1:3] 1 1 1 1 1 1 1 1 1 1 ...
  .. ..- attr(*, "dimnames")=List of 2
  .. .. ..$ : NULL
  .. .. ..$ : chr [1:3] "" "ninth_hour" "tenth_hour"
  ..$ : num [1:94, 1:11] 1 1 1 1 1 1 1 1 1 1 ...
 $ net.result: num [1:94, 1] 0.684 -0.255 -1.588 0.89 1.091 ...


In [ ]:
set.seed(12345)
# two layer neural network
model_two_hidden_layers = function(input_set,first_hidden_layer_neurons,secoend_hidden_layer_neurons,activation_fun) {
#nuerel network architechure    
autoregressive_nn = neuralnet(eleventh_hour ~ input_set,
data=df_train, hidden=c(first_hidden_layer_neurons,secoend_hidden_layer_neurons), linear.output=TRUE,act.fct=activation_fun)

# train_results = compute(nn_model_true,gbp_test_data[,3:4])

# truthcol = gbp_exchange_full[401:491,3]$usd_eur

# predcol = unnormalize(train_results$net.result,gbp_min_train_data,
# gbp_max_train_data)[,1]
# relevant_pred_stat(truthcol,predcol,
# "Two Hidden Layers") %>%
# mutate(hiddel_layers = paste0(hidden, " and ",sec_hidden),
# input_set = "A") %>%
# filter(.metric != "rsq")
}

In [ ]:
autoregressive_nn = neuralnet(eleventh_hour ~ ninth_hour+tenth_hour,
data=df_train, hidden=c(5,5), linear.output=TRUE,act.fct="logistic")

In [ ]:
train_results = compute(autoregressive_nn,df_test[,c(1,2)])

In [ ]:
truthcol = df_test[]

ERROR: Error in df_test$eleventh_hour: $ operator is invalid for atomic vectors


In [ ]:
truthcol

[1] 0.7259547

In [ ]:
data(algae)
normData <- scale(algae[,4:12])
t <- rpartXse(a1 ~ .,as.data.frame(normData[1:100,]))
normPs <- predict(t,as.data.frame(normData[101:nrow(normData),]))
ps <- unscale(normPs,normData)
## Not run: 
#  plot(algae[101:nrow(algae),'a1'],ps)
# ## End(Not run)
